In [43]:
import pandas as pd                     # for dataframes' manipulation
from pandas import DataFrame            # for creating dataframes
import numpy as np                      # for arrays

from itertools import chain

from collections import Counter         # for counting elements 
from datetime import datetime           #for actual date
import re                               # !!! relativní Novinka - regular expressions
from time import sleep                  # for sleeping (slowing down) inside a function
import random                           # for random number (sleeping)
import math                             # Round float
import time                             # Time measuring
import itertools                        # for unlisting nested lists
import sys
import openpyxl

##### Scraping ############
import requests                         # for robots check
from bs4 import BeautifulSoup           # for parsing
from selenium import webdriver          # for browsers control
import json                             # for Requests

In [44]:
# 1 = Scraping ################################################################################################################################

def get_soup_elements(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):  
    
    browser = webdriver.Chrome()
    
    ##########################################
    # 1. Volba Prodej/Pronájem, Byty/Domy,                 --Aukce/Bez Aukce (jen pro Prodeje) zatím nechávám být, cpe se mi to doprostřed url
    ##########################################   
    
    url_x = r"https://www.sreality.cz/hledani"             
    url = url_x + "/" +  typ_obchodu + "/" +  typ_stavby

    ##########################################
    # 2. načtení webu
    ##########################################
    
    browser.get(url)    # (url).text ??
    sleep(random.uniform(1.0, 1.5))
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,'lxml') # "parser" ??
    
    elements = []    
    
    for link in soup.findAll('a', attrs={'href': re.compile("^/detail/")}):       # !!!!!!!!!!!!!!!!! změněno, protože H2 neobsahovalo všechny věci, jen nadpisek.
        link = link.get('href')   
        elements.append(link)     
    elements = elements[0::2]   

    ##########################################
    # 3. zjištění počtu listů - mělo by být optional, ale nevadí
    ##########################################
    records = soup.find_all(class_ ='numero ng-binding')[1].text
    records = re.split(r'\D', str(records))                         
    records = ",".join(records).replace(",", "")
    records = int(records)
    max_page = math.ceil(records / 20)   
    print("----------------")
    print("Scrapuji: " + str(typ_obchodu) + " " + str(typ_stavby))
    print("Celkem inzerátů: " + str(records))
    print("Celkem stránek: " + str(max_page))
    
    ##########################################
    # 4. nastavení počtu stránek  -mělo být víc promakané
    ##########################################
    if pages == 999:      # (NE)Speciální případ, chci všechny inzeráty - standardně tu bude asi kolem 600 stránek max, takže 999 je volné k použití
        pages = max_page
    
    print("Scrapuji (pouze) " + str(pages) + " stran.")
    print("----------------")
    
    ##########################################
    # 5. Scrapping zbylých listů - naštěstí v jednom okně
    ##########################################    
    
    for i in range(pages-1):   
        i = i+2
        
        sys.stdout.write('\r'+ "Strana " + str(i-1) + " = " + str(round(100*(i-1)/(pages), 2)) + "% progress. Zbývá cca: " + str(round(random.uniform(3.4, 3.8)*(pages-(i-1)), 2 )) + " sekund.")    # Asi upravím čas, na rychlejším kabelu v obýváku je to občas i tak 3 sec :O

        url2 = url + "?strana=" + str(i)
        browser.get(url2)

        sleep(random.uniform(1.0, 1.5))

        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup2 = BeautifulSoup(innerHTML,'lxml') 
        
        elements2 = []
        
        for link in soup2.findAll('a', attrs={'href': re.compile("^/detail/prodej/")}):  
            link = link.get('href') 
            elements2.append(link)  
   
        elements2 = elements2[0::2]  
        
        elements = elements + elements2     # tyto se už můžou posčítat, naštěstí, řpedtím než z nich budeme dělat elems = prvky třeba jména

    
    browser.quit()   
    
    return elements


In [45]:
#  2 = Získání URLS ###############################################################################################################################

def elements_and_ids(x):
    
    elements = pd.DataFrame({"url":x})

    def get_id(x):
        x = x.split("/")[-1]
        return x

    #elements["url_id"] = elements["url"].apply(get_id)
    
    len1 = len(elements)
    #Přidáno nově, v tuto chvíli odmažu duplikáty a jsem v pohodě a šetřím si čas dál.
    elements = elements.drop_duplicates(subset = [ "url"], keep = "first", inplace = False)   
    len2 = len(elements)                                                                             
                                                                                                      
    print("-- Vymazáno " + str(len1-len2) + " záznamů kvůli duplikaci.")
    return elements

In [46]:
# Parametry:
# "prodej"/ "pronájem"
# "byty"/"domy"
# pages = 1- X, případně 999 = Všechny strany !

def scrap_all(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):
    
    # Scrapni data - hezky komunikuje = cca 50 min
    data = get_soup_elements(typ_obchodu = typ_obchodu, typ_stavby = typ_stavby, pages = pages)
    print( "1/8 Data scrapnuta, získávám URLs.")
    
    # 2 = Získání URLS
    data = elements_and_ids(data)
    data.to_excel(r"a1_URLs_prodej_byty.xlsx")
    print( "2/8 Získány URL, nyní získávám Souřadnice, Ceny a Popis - několik minut...")
    

    return data

In [50]:
data = scrap_all(pages=25)

----------------
Scrapuji: prodej byty
Celkem inzerátů: 20626
Celkem stránek: 1032
Scrapuji (pouze) 25 stran.
----------------
Strana 24 = 96.0% progress. Zbývá cca: 3.63 sekund..1/8 Data scrapnuta, získávám URLs.
-- Vymazáno 1240 záznamů kvůli duplikaci.
2/8 Získány URL, nyní získávám Souřadnice, Ceny a Popis - několik minut...


In [ ]:
data1 = data

In [ ]:
data[['null','detail', 'prodej','byt','velikost','lokace','id']] = data.url.str.split("/", expand = True)
data = data.drop(['url','null', 'detail','prodej','byt','id'], axis=1)
data.to_csv('sreality.csv')

In [57]:
data.to_csv('sreality.csv')

In [60]:
import psycopg2
conn = psycopg2.connect(host='localhost',
port = '5432',
user = 'postgres',
password = '159357lol',
dbname = 'postgres')

cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS sreality')

create_script= "CREATE TABLE IF NOT EXISTS sreality (ID INT,Velikost VARCHAR(255), Lokace VARCHAR(255))"
cur.execute(create_script)

with open('sreality.csv', 'r') as f:
    # Notice that we don't need the csv module.
    next(f) # Skip the header row.
    cur.copy_from(f, 'sreality', sep=',')

conn.commit()
conn.close()
cur.close()

UndefinedTable: relation "users" does not exist
